# 受講者の演習環境をバックアップします
---

受講者の演習環境をバックアップしたい場合に使用します。バックアップするファイル/フォルダを指定してバックアップします。

# バックアップする受講者を指定します

バックアップ対象の受講者をメールアドレスで指定します。

バックアップファイルは、`メールアドレス-backup.tgz` のファイル名で、後述するバックアップ格納先フォルダに保存します。
```
STUDENT_LIST = '''
student-a01@example.com
student-a02@example.com
'''
```

**次のセルを実行して、バックアップ対象の受講者のメールアドレスを設定してください**

In [ ]:
STUDENT_LIST = '''
student-a01@example.com
student-a02@example.com
'''

# バックアップの格納先フォルダを指定します

* バックアップファイルを格納するフォルダを指定します。
* ホームディレクトリからの相対パスで指定します。
* 空に設定するとホームディレクトリ直下にバックアップファイルが格納されます。
* フォルダが存在しない場合は、自動的に作成されます。
* フォルダ内に同一受講者のバックアップが存在する場合、バックアップ時に上書きされます。

**次のセルを実行して、バックアップ格納先フォルダを指定して下さい。**

In [ ]:
BACKUP_DIR='backup/0123'

# バックアップから除外するファイル/フォルダを指定します

受講者のホームディレクトリ以下で、受講者が作成した全てのノートブック/その他ファイルがバックアップの対象となります。ただし、次のファイル/フォルダはバックアップにからは除外されます。
- `info/` フォルダ
- `private-info/` フォルダ
- `textbook/` フォルダ
- `tools/` フォルダ
- `.(dot)` で始まる全てのファイルとフォルダ

他にバックアップから除外したいファイル/フォルダがある場合は、`BACKUP_EXCLUDE` に改行区切りで列挙してください。ワイルドカードの `*` が使用できます。

```
BACKUP_EXCLUDE = '''
*.tmp
*.gz
test/
user-folder/sub-folder/*.ipynb
user-folder/hoge.txt
'''
```

**次のセルを実行して、除外するファイル/フォルダを設定してください。**

In [ ]:
BACKUP_EXCLUDE = '''
*.tmp
*.gz
test/
user-folder/sub-folder/*.ipynb
user-folder/hoge.txt
'''

# バックアップを実行します

設定した内容で、バックアップを実行します。

バックアップファイルは、指定した格納先フォルダに、受講者毎のファイルとして格納されます。

In [ ]:
import os, sys, re, hashlib, string, tempfile, shutil, subprocess, fnmatch

def get_username_from_mail_address(mail_address):
    # Convert to lower and remove characters except for alphabets and digits
    wk = mail_address.split('@')
    local_part = wk[0].lower()
    result = re.sub(r'[^a-zA-Z0-9]', '', local_part)
    # Add top 6bytes of hash string
    md5 = hashlib.md5()
    md5.update(mail_address.encode('us-ascii'))
    h = md5.hexdigest()[0:6]
    result += 'x'
    result += h;
    return result;

def ignore_files(path, names):
    dir = re.sub(r'^/home/jupyter/workspace/[^/]+(/|$)', '', path)
    pats = set(map(lambda x: x[1], filter(lambda x: (x[0] is None) or (dir == x[0]), exclude_patterns)))
    ignore_set = shutil.ignore_patterns(*pats)(path, names)
    #for ent in ignore_set:
    #    print('DEBUG: ignored file {}/ {}'.format(path, ent))
    return ignore_set

#
target_students=set()
for line in STUDENT_LIST.splitlines():
    s = re.sub(r'#.*$', '', line).strip()
    if(0 < len(s)):
        target_students.add(s)
        
#
EXCLUDE_SYS={'.*','info/','private-info/','textbook/','tools/'}
exclude_patterns=set()
for line in set(BACKUP_EXCLUDE.splitlines()).union(EXCLUDE_SYS):
    s = re.sub(r'#.*$', '', line).strip()
    if(0 < len(s)):
        if '/' in s:
            s = s.rstrip('/')
            exclude_patterns.add((os.path.dirname(s), os.path.basename(s)))
        else:
            exclude_patterns.add((None, s))

#
backup_dir_path = os.path.expanduser('~/{}'.format(os.path.normpath(BACKUP_DIR)))
os.makedirs(backup_dir_path, exist_ok=True)

#
with tempfile.TemporaryDirectory() as tmp_dir:
    for student in target_students:
        print('受講者[{}]のバックアップを作成します'.format(student))
        user_name = get_username_from_mail_address(student)
        user_dir = '/home/jupyter/workspace/{}'.format(user_name)
        if not os.path.isdir(user_dir):
            print('WARNING: {}: 受講者フォルダが見つかりません'.format(student), file=sys.stderr)
            print('-> 失敗')
            continue
        tmp_user_dir = '{}/{}'.format(tmp_dir, user_name)
        try:
            shutil.copytree(user_dir, tmp_user_dir, ignore=ignore_files)
        except shutil.Error as ex:
            for ee in ex.args[0]:
                print('WARNING: {}'.format(ee[2]), file=sys.stderr)
        # archive it
        archive_file_path = '{}/{}-backup.tgz'.format(backup_dir_path, student)
        try:
            tarcmd = 'tar -cz -C "{}" -f "{}" .'.format(tmp_user_dir, archive_file_path)
            cp = subprocess.run(tarcmd, shell=True, cwd=tmp_user_dir)
            if cp.returncode == 0:
                print('-> バックアップ成功: {}'.format(archive_file_path))
            else:
                print('ERROR: tar コマンドが失敗しました', file=sys.stderr)
        except OSError as e:
            print('ERROR: tar コマンドが実行できません：{}'.format(e), file=sys.stderr)
        print('')

print('バックアップ処理が完了しました。')